In [1]:
import pandas as pd

import sys
sys.path.append('..')

# ID Creation

In [ ]:
import identify as id

In [13]:
test_buildings = pd.read_pickle("random_sample_nona_matched.pkl").reset_index()
test_buildings = test_buildings[['geometry', 'utm_area', 'rand_geometry', 'rand_utm_area',]]

In [16]:
#test_buildings[f'rand_locid'] = test_buildings.apply(lambda x: id.identify(x.rand_geometry), axis=1) 
test_buildings[f'locid'] = test_buildings.apply(lambda x: id.identify(x.geometry), axis=1) 

# Matching

In [ ]:
from operations import match

In [ ]:
test_buildings[f'match_t{tol}_l{lvl}'] = test_buildings.apply(lambda x: match(x.locid, x.rand_locid),axis = 1)